# Final assignment 
## Language switching experiment

### Things to do:
- Create a _PsychoPy experiment_ to collect _response time_ data
- Store the RTs and accuracy in a _csv file_
- _Plot_ the data using seaborn
- _Analyze_ the data using PyMC3 or Statsmodels
- __Commit__ all of your changes regularly 

## 0. Import all necessary libraries
Import sounddevice ad given in the _hints_ file

In [1]:
import sounddevice as sd
import soundfile as sf
from psychopy import prefs
prefs.general['audioLib'] = ['pygame']
from psychopy import core, visual, event, gui, data, clock, logging
from psychopy.tools.filetools import fromFile, toFile
from psychopy.constants import (NOT_STARTED, STARTED, PLAYING, PAUSED, STOPPED, FINISHED, PRESSED, RELEASED, FOREVER)
import numpy as np
import os 
import sys
import pandas as pd
import seaborn as sns
import csv

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


## 1. Creating the experiment
### 1.1. Stimuli

The stimuli will consist of pictures, presented on one side of the screen. When a pictures is presented on the _left_ side of the screen, the picture needs to be named in L1. When a picture is presented on the _right_ side of the screen, the picture needs to be named in the L2. 

The stimuli are taken from the MultiPic project (Duñabeitia, J. A., Crepaldi, D., Meyer, A. S., New, B., Pliatsikas, C., Smolka, E., & Brysbaert, M. (2018). MultiPic: A standardized set of 750 drawings with norms for six European languages. Quarterly Journal of Experimental Psychology, 71(4), 808–816. https://doi.org/10.1080/17470218.2017.1310261). 


Then I created a PsychoPy experiment called `switch.py`. First of all, to open a window:

In [10]:
'''
from psychopy import core, visual, event 
win = visual.Window(color = 'white', fullscr = True) #Open a window with a white background, full screen instead of pixels 
core.wait(5)  
win.close()  
'''

"\nfrom psychopy import core, visual, event \nwin = visual.Window(color = 'white', fullscr = True) #Open a window with a white background, full screen instead of pixels \ncore.wait(5)  # wait for 5 seconds so we can look at the window\nwin.close()  # close the window\n"

I would like to include an introduction screen with written instructions for the participants. I did this using separate commands:

In [ ]:
'''
# prepare a text stimulus
text = visual.TextStim(win, text='Welcome to this experiment', color='Black', font = 'arial')
text.draw()  # draw the stimulus to the back buffer
win.flip()  # flip the back buffer with our stimulus to the front
core.wait(5)  # wait     

text.setText('In this experiment, you will get to see pictures. Your task is to name the pictures as quickly as possible.')
text.draw()  # draw to back buffer again
win.flip()
core.wait(8)  # wait for 8 seconds so we can look at the window with our text
    
text.setText('However, there is a catch.')
text.draw() 
win.flip()
core.wait(5)  
    
etc.
'''

But then converted it into a function. Also, instead of having a fixed amount of time to read it, I tried to have participants press a key to continue:

In [ ]:
'''
#Function for instruction test
def instr(win, myText):
    text.setText(myText)
    text.draw()  # draw to back buffer again
    win.flip()
    return

#Function to present the instruction text and to continue 
def presentInstr(win, myText):
    event.clearEvents()
    while len(event.getKeys())==0:
        instr(win, myText)
    return

presentInstr(win, 'Welcome to this experiment.\n\nPress any key to continue.')

Now, the actual stimuli need to be presented. There are _seven_ (or more) pictures that need to be presented on the left or right side of the screen. Pictures on the left side need to be named in English and are therefore the "EN-trials". Pictures on the right side of the screen will need to be named in Dutch (i.e., all other trials). The pictures need to stay on the screen for as long as it takes to name them. This, however, is something I will try in the subsequent part of the assignment. Therefore:

In [ ]:
#Function for generating the trials
'''
def trial(win, pict, lang):
    if lang == 'EN': 
        LeftIm.setImage(pict)
        LeftIm.draw()
    else:
        #i.e., the Dutch trials 
        RightIm.setImage(pict)
        RightIm.draw() 
        
    win.flip()
    core.wait(2) #core.wait is until I figure the voicekey out
    return
'''

In [4]:
#This is the code for the switch experiment 

#First import the necessary libraries 
#Import sounddevice as given in the HINTS file
import sounddevice as sd
import soundfile as sf
from psychopy import prefs
prefs.general['audioLib'] = ['pygame']
from psychopy import core, visual, event, gui, data, clock, logging
from psychopy.tools.filetools import fromFile, toFile
from psychopy.constants import (NOT_STARTED, STARTED, PLAYING, PAUSED, STOPPED, FINISHED, PRESSED, RELEASED, FOREVER)
import numpy, random #for handling various numerical/mathematical functions
import os 
import sys #to get file system encoding

#Ensure that relative paths start from same directory as script
_thisDir = 'C:/Users/saski/mpi-python-intro/Session6/'

#Information on the experiment session
expName = 'Switch_experiment'
expInfo = {'Participant': '', 'Gender':'', 'Age': ''}
dlg = gui.DlgFromDict(dictionary = expInfo, title = expName)
if dlg.OK == False:
    core.quit() #Participant pressed cancel
expInfo['date'] = data.getDateStr() #add a simple timestamp
expInfo['expName'] = expName

#Data file name stem = absolute path + name; later add .csv
filename = _thisDir + os.sep + u'data/%s_%s_%s' % (expInfo['Participant'], expName, expInfo['date'])

# thisExp = data.ExperimentHandler(name = expName, version = '', extraInfo = expInfo, runtimeInfo = None, originPath = None, savePickle = True, saveWideText = True, dataFileName = filename)


#You can implement and 'escape' to quit the experiment
endExpNow = False 

#Setting up the microphone
sd.default.samplerate = 48000 #record at 48000 samples per second
sd.default.channels = 1 #record in mono

#Create window and stimuli
win = visual.Window([1920, 1080], color = 'white', monitor = 'testMonitor', units = 'cm') #Open a window with a white background
text = visual.TextStim(win, None, color='black', font = 'arial') #Prepare a text stimulus
LeftIm = visual.ImageStim(win, None) #Create the left images
LeftIm.setPos((-8,0)) #Define the position on the screen
RightIm = visual.ImageStim(win, None) #Create the right images 
RightIm.setPos((8,0)) #Define the position on the screen

#Clocks to keep track of time
globalClock = core.Clock()
trialClock = core.Clock()

#The needed functions
#Function for instruction test
def instr(win, myText):
    text.setText(myText)
    text.draw()  # draw to back buffer again
    win.flip()
    return

#Function to present the instruction text and to continue 
def presentInstr(win, myText):
    event.clearEvents()
    while len(event.getKeys())==0:
        instr(win, myText)
    return

#Function for generating the trials
def trial(win, pict, lang, sound):
    t = 0
    trialClock.reset()
    if lang == 'EN': 
        LeftIm.setImage(pict)
        LeftIm.draw()
    else:
        #i.e., the Dutch trials 
        RightIm.setImage(pict)
        RightIm.draw() 
        
    win.flip()
    t = trialClock.getTime()
    sample = sd.rec(4 * sd.default.samplerate) #record for four seconds
    sd.wait() #finish recording before moving on
    sf.write(sound, sample, sd.default.samplerate)
    
    if endExpNow or event.getKeys(keyList=['escape']):
        core.quit()
    
    return

###INSTRUCTIONS###
presentInstr(win, 'Welcome to this experiment.\n\nPress any key to continue.')
presentInstr(win, 'You will get to see pictures. \n\nYour task is to name the pictures as quickly as possible.')
presentInstr(win, 'However, there is a catch.')
presentInstr(win, 'If you see a picture on the LEFT side of the screen, name the picture in English.')
presentInstr(win, 'If you see a picture on the RIGHT side of the screen, name the picture in Dutch.')
presentInstr(win, 'Ready? \n\nPress a key to start!')

###EXPERIMENT###
#This can probably be done in a better way
trial(win, u'stimuli/1.png', 'EN', u'recordings/%s_1.wav' % (expInfo['Participant'])) #Added directories to save it in a separate folder
trial(win, u'stimuli/2.png', 'EN', u'recordings/%s_2.wav' % (expInfo['Participant']))
trial(win, u'stimuli/3.png','EN', u'recordings/%s_3.wav' % (expInfo['Participant']))
trial(win, u'stimuli/4.png', 'NL', u'recordings/%s_4.wav' % (expInfo['Participant']))
trial(win, u'stimuli/5.png', 'NL', u'recordings/%s_5.wav' % (expInfo['Participant']))
trial(win, u'stimuli/6.png','EN', u'recordings/%s_6.wav' % (expInfo['Participant']))
trial(win, u'stimuli/7.png', 'NL', u'recordings/%s_7.wav' % (expInfo['Participant']))


#Cleanup
win.close()
core.quit() 

1163.0896 	ERROR 	Couldn't find image stimuli/1.png; check path? (tried: C:\Users\saski\mpi-python-intro\Session6\recordings\stimuli\1.png)


OSError: Couldn't find image stimuli/1.png; check path? (tried: C:\Users\saski\mpi-python-intro\Session6\recordings\stimuli\1.png)

### 1.2 Responses 

The participants need to _name_ the pictures, meaning that the responses will consist of a voice key. I tried to figure out how to do this, but wasn't able to. Instead, I used _sound device_ to record what the participants say for a fixed time, and then extract the reaction times like this:

In [3]:

path = 'C:/Users/saski/mpi-python-intro/Session6/recordings/' #Define the right directory
os.chdir(path)

rec = [] #Create an empty list for recordings
listRec = os.listdir('.') 

#Create a list of all the .wav files in the directory
for file in listRec:
    if file.endswith(".wav"):
         rec.append(file)

for files in listRec:
    recordings, samplerate = sf.read(files) #Read all .wav files 
    energy = np.square(recordings) #create a variable "energy" of the squared recordings-array
    Emax = np.max(energy)
    energyNorm = (energy/Emax) #Normalize these values
    index = [index for index, value in enumerate(energyNorm) if value > 0.2] #Ok so this is quite unprecise and there are multiple solutions to this problem (e.g., comparing multiple indices, getting rid of noise beforehand) , but for this exercise I will keep it like this. 
    RT = index[0]/samplerate #This is the reaction time
    print(RT) #Sanity check
    
# RTdata = "RTdata.csv" #where files need to be downloaded to
# RTcsv = (RTdata, "w")

# RT.saveAsWideText(filename + '.csv')

with open(filename + '.csv', 'w') as RT_file:
        
        # use str.join() to turn the list of lines into a single string again before we write to file
#         chapter_text = '\n'.join(lines)
        
        # write to file
        RT_file.write(RT)
    

1.3002916666666666
2.7606041666666665
1.2708541666666666
0.2765
2.1641875
0.7665208333333333
1.2252708333333333
3.174


NameError: name 'filename' is not defined

### 1.3 Saving the results

Firstly, I'd like to get some participant information, which they can fill in in an dialogue box (`dlg`). This dialogue box is presented to the participant. 

In [ ]:
'''
#Information on the experiment session
expName = 'Switch_experiment'
expInfo = {'Participant': '', 'Gender':'', 'Age': ''}
dlg = gui.DlgFromDict(dictionary = expInfo, title = expName)
if dlg.OK == False:
    core.quit() #Participant pressed cancel
expInfo['date'] = data.getDateStr() #add a simple timestamp
expInfo['expName'] = expName
'''

After each trial, a `.txt` file is created to log the experiment:

In [ ]:
'''
#Save log file
logFile = logging.LogFile(filename + '.log', level = logging.EXP)
logging.console.setLevel(logging.WARNING) #this outputs to the screen, not a file
'''

The recordings need to be made and saved in the right directory. The _sound device_ is used to make the recordings, and _sound file_ to write the recordings in the right folder. 

In [5]:
'''
#Function for generating the trials
def trial(win, pict, lang, sound):
    t = 0
    trialClock.reset()
    if lang == 'EN': 
        LeftIm.setImage(pict)
        LeftIm.draw()
    else:
        #i.e., the Dutch trials 
        RightIm.setImage(pict)
        RightIm.draw() 
        
    win.flip()
    t = trialClock.getTime()
    sample = sd.rec(4 * sd.default.samplerate) #record for four seconds
    sd.wait() #finish recording before moving on
    sf.write(sound, sample, sd.default.samplerate)
    
    if endExpNow or event.getKeys(keyList=['escape']):
        core.quit()
    
    return
'''

'C:\\Users\\saski\\mpi-python-intro\\Session6'

### 1.4 Try it out

In [18]:
#Try it out
import subprocess as sp

sp.run(['python', 'switch.py'])

CompletedProcess(args=['python', 'switch.py'], returncode=0)

## 2.0 Plot the data 

## 3.0 "Analyze" the data